## Import required libraries

In [1]:
import requests
import pandas as pd
import numpy as np

# Part 1: Getting and cleaning the data

## Scraping Wikipedia page to get Toronto Neighbourhoods dataframe

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
r = requests.get(url) #do I need custom header here?

In [4]:
dfs=pd.read_html(r.text, header=0)

In [5]:
toronto_neis = dfs[0]
toronto_neis.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
toronto_neis.shape

(180, 3)

## Let's clean the dataframe

In [7]:
toronto_neis['Borough'].value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

In [8]:
toronto_neis = toronto_neis[toronto_neis.Borough != 'Not assigned']
toronto_neis.shape

(103, 3)

In [9]:
toronto_neis.loc[toronto_neis['Neighbourhood'] == 'Not Assigned']

,Postal Code,Borough,Neighbourhood


In [10]:
toronto_neis.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Part 2: Geocoding

In [11]:
geo_csv = pd.read_csv('Geospatial_Coordinates.csv')

In [12]:
geo_csv.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
toronto_neis_geocoded = pd.merge(toronto_neis, geo_csv, on='Postal Code')
toronto_neis_geocoded.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Part 3: Clustering